# SQL query to create a fact_calls table

In [ ]:
-- Creating a staging table for senders with call details
WITH stg_senders AS (
  SELECT call_id, call_date, call_time, length_seconds, status, calls.sender_id, name AS sender_name, username AS sender_username,
  phone AS sender_phone
  FROM senders
  JOIN calls ON senders.sender_id = calls.sender_id),

-- Creating a staging table for receivers with call details
stg_receivers AS (
  SELECT call_id AS call_id2, call_date AS call_date2, call_time AS call_time2, length_seconds AS length_seconds2, status AS status2, calls.receiver_id, 
  name AS receiver_name, username AS receiver_username, phone AS receiver_phone
  FROM receivers
  JOIN calls ON receivers.receiver_id = calls.receiver_id),

-- Joining both sender and receiver staging tables on call_id
stg_calls AS (
  SELECT *
  FROM stg_senders
  JOIN stg_receivers ON stg_receivers.call_id2 = stg_senders.call_id),

-- Creating a dimension table for senders
dim_senders AS (
  SELECT DISTINCT sender_id, sender_name, sender_username, sender_phone
  FROM stg_calls),

-- Removing sender_name, sender_username, and sender_phone from the staging table
stg_calls2 AS (
  SELECT call_id, call_date, call_time, length_seconds, status, sender_id, call_date2, call_time2, length_seconds2, status2, receiver_id, receiver_name, receiver_username, 
  receiver_phone
  FROM stg_calls),

-- Creating a dimension table for receivers
dim_receivers AS (
  SELECT DISTINCT receiver_id, receiver_name, receiver_username, receiver_phone
  FROM stg_calls2),

-- Removing receiver_name, receiver_username, and receiver_phone from the staging table
stg_calls3 AS (
  SELECT call_id, call_date, call_time, length_seconds, status, sender_id, receiver_id
  FROM stg_calls2),

-- Creating the fact_calls table by selecting required columns from the final staging table
fact_calls AS (
  SELECT call_id, call_date, call_time, length_seconds, status, sender_id, receiver_id
  FROM stg_calls3)

-- Selecting rows from the fact_calls table
SELECT *
FROM fact_calls
LIMIT 5;

# SQL query to create a dim_senders table

In [ ]:
-- Creating a staging table for senders with call details and device information
WITH stg_senders AS (
  SELECT call_id, call_date, call_time, length_seconds, status, calls.sender_id, name AS sender_name, username AS sender_username,
  phone AS sender_phone, product_name AS sender_product_name, software_version AS sender_software_version
  FROM senders
  JOIN calls ON senders.sender_id = calls.sender_id
  JOIN devices ON senders.sender_id = devices.device_id),

-- Creating the dim_call_senders table
dim_call_senders AS (
  SELECT DISTINCT call_id, sender_id, sender_name, sender_username, sender_phone, call_date, call_time, length_seconds, status,
  sender_product_name, sender_software_version
  FROM stg_senders)

-- Selecting rows from the dim_call_senders table
SELECT *
FROM dim_call_senders
LIMIT 5;

# SQL query to create a dim_receivers table

In [ ]:
-- Creating a staging table for receivers with call details and device information
WITH stg_receivers AS (
  SELECT call_id, call_date, call_time, length_seconds, status, calls.receiver_id, name AS receiver_name, username AS receiver_username,
  phone AS receiver_phone, product_name AS receiver_product_name, software_version AS receiver_software_version
  FROM receivers
  JOIN calls ON receivers.receiver_id = calls.receiver_id
  JOIN devices ON receivers.receiver_id = devices.device_id),

-- Creating the dim_call_receivers table
dim_call_receivers AS (
  SELECT DISTINCT call_id, receiver_id, receiver_name, receiver_username, receiver_phone, call_date, call_time, length_seconds, status,
  receiver_product_name, receiver_software_version
  FROM stg_receivers)

-- Selecting rows from the dim_call_receivers table
SELECT *
FROM dim_call_receivers
LIMIT 5;

-- Create the fact_device_history table
CREATE TABLE fact_device_history (
  device_id INT,
  gps_date TEXT,
  gps_time TEXT,
  latitude INT,
  longitude INT,
  latitude_direction VARCHAR(15),
  longitude_direction VARCHAR(15),
  lock_date TEXT,
  lock_time TEXT,
  locked BOOLEAN,
  power_date TEXT,
  power_time TEXT,
  powered_on BOOLEAN,
  bt_connection_date TEXT,
  bt_connection_time TEXT,
  cb_connection_date TEXT,
  cb_connection_time TEXT,
  charge_date TEXT,
  charge_time TEXT,
  login_date TEXT,
  login_time TEXT,
  attempt_number INT,
  successful BOOLEAN,
  login_method VARCHAR(25),
  contact_num INT,
  first_name VARCHAR(50),
  last_name VARCHAR(50),
  date_added TEXT,
  time_added TEXT,
  home_phone VARCHAR(15),
  cell_phone VARCHAR(15),
  work_phone VARCHAR(15),
  email VARCHAR(50),
  on_body_date TEXT,
  on_body_time TEXT,
  on_body BOOLEAN
);

-- Insert location data
INSERT OR IGNORE INTO fact_device_history (
  device_id, gps_date, gps_time, latitude, longitude, latitude_direction, longitude_direction
)
SELECT
  device_id, gps_date, gps_time, latitude, longitude, latitude_direction, longitude_direction
FROM location;

INSERT OR IGNORE INTO fact_device_history (device_id)
SELECT device_id
FROM lock_status;

UPDATE fact_device_history
SET lock_date = (SELECT lock_date FROM lock_status WHERE fact_device_history.device_id = lock_status.device_id),
    lock_time = (SELECT lock_time FROM lock_status WHERE fact_device_history.device_id = lock_status.device_id),
    locked = (SELECT locked FROM lock_status WHERE fact_device_history.device_id = lock_status.device_id);

-- For power_status
INSERT OR IGNORE INTO fact_device_history (device_id)
SELECT device_id
FROM power_status;

UPDATE fact_device_history
SET power_date = (SELECT power_date FROM power_status WHERE fact_device_history.device_id = power_status.device_id),
    power_time = (SELECT power_time FROM power_status WHERE fact_device_history.device_id = power_status.device_id),
    powered_on = (SELECT powered_on FROM power_status WHERE fact_device_history.device_id = power_status.device_id);

-- For bluetooth_connections
INSERT OR IGNORE INTO fact_device_history (device_id)
SELECT device_id
FROM bluetooth_connections;

UPDATE fact_device_history
SET bt_connection_date = (SELECT connection_date FROM bluetooth_connections WHERE fact_device_history.device_id = bluetooth_connections.device_id),
    bt_connection_time = (SELECT connection_time FROM bluetooth_connections WHERE fact_device_history.device_id = bluetooth_connections.device_id);

-- For cable_connections
INSERT OR IGNORE INTO fact_device_history (device_id)
SELECT device_id
FROM cable_connections;

UPDATE fact_device_history
SET cb_connection_date = (SELECT connection_date FROM cable_connections WHERE fact_device_history.device_id = cable_connections.device_id),
    cb_connection_time = (SELECT connection_time FROM cable_connections WHERE fact_device_history.device_id = cable_connections.device_id);

-- For charge_status
INSERT OR IGNORE INTO fact_device_history (device_id)
SELECT device_id
FROM charge_status;

UPDATE fact_device_history
SET charge_date = (SELECT charge_date FROM charge_status WHERE fact_device_history.device_id = charge_status.device_id),
    charge_time = (SELECT charge_time FROM charge_status WHERE fact_device_history.device_id = charge_status.device_id);

-- For logins
INSERT OR IGNORE INTO fact_device_history (device_id)
SELECT device_id
FROM logins;

UPDATE fact_device_history
SET login_date = (SELECT login_date FROM logins WHERE fact_device_history.device_id = logins.device_id),
    login_time = (SELECT login_time FROM logins WHERE fact_device_history.device_id = logins.device_id),
    attempt_number = (SELECT attempt_number FROM logins WHERE fact_device_history.device_id = logins.device_id),
    successful = (SELECT successful FROM logins WHERE fact_device_history.device_id = logins.device_id),
    login_method = (SELECT login_method FROM logins WHERE fact_device_history.device_id = logins.device_id);

-- For contacts
INSERT OR IGNORE INTO fact_device_history (device_id)
SELECT device_id
FROM contacts;

UPDATE fact_device_history
SET contact_num = (SELECT contact_num FROM contacts WHERE fact_device_history.device_id = contacts.device_id),
    first_name = (SELECT first_name FROM contacts WHERE fact_device_history.device_id = contacts.device_id),
    last_name = (SELECT last_name FROM contacts WHERE fact_device_history.device_id = contacts.device_id),
    date_added = (SELECT date_added FROM contacts WHERE fact_device_history.device_id = contacts.device_id),
    time_added = (SELECT time_added FROM contacts WHERE fact_device_history.device_id = contacts.device_id),
    home_phone = (SELECT home_phone FROM contacts WHERE fact_device_history.device_id = contacts.device_id),
    cell_phone = (SELECT cell_phone FROM contacts WHERE fact_device_history.device_id = contacts.device_id),
    work_phone = (SELECT work_phone FROM contacts WHERE fact_device_history.device_id = contacts.device_id),
    email = (SELECT email FROM contacts WHERE fact_device_history.device_id = contacts.device_id);

-- For on_body_status
INSERT OR IGNORE INTO fact_device_history (device_id)
SELECT device_id
FROM on_body_status;

UPDATE fact_device_history
SET on_body_date = (SELECT on_body_date FROM on_body_status WHERE fact_device_history.device_id = on_body_status.device_id),
    on_body_time = (SELECT on_body_time FROM on_body_status WHERE fact_device_history.device_id = on_body_status.device_id),
    on_body = (SELECT on_body FROM on_body_status WHERE fact_device_history.device_id = on_body_status.device_id);



